[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/asteroid-team/asteroid/blob/master/notebooks/04_ProcessLargeAudioFiles.ipynb)

### Process large audio files
When running inference with a separation/enhancement model, long wav files can
cause CPU freeze/GPU OOM. Asteroid provides the `LambdaOverlapAdd` module that
decomposes the input signal in overlapping chunks, apply the desired lambda function
and re-combines the segment using overlap-add.
Pair it with a pretrained model to enhance your long files !

In [3]:
import torch

from asteroid import ConvTasNet
from asteroid.dsp.overlap_add import LambdaOverlapAdd


nnet = ConvTasNet(n_src=2)
continuous_nnet = LambdaOverlapAdd(
		nnet=nnet,  # function to apply to each segment.
		n_src=2,  # number of sources in the output of nnet
		window_size=64000,  # Size of segmenting window
		hop_size=None,  # segmentation hop size
		window="hann",  # Type of the window (see scipy.signal.get_window
		reorder_chunks=True,  # Whether to reorder each consecutive segment.
		enable_grad=False,  # Set gradient calculation on of off (see torch.set_grad_enabled)
)

# Instantiate long wav (1min) that would OOM
wav = torch.randn(1, 1, 500000)
out_wavs = continuous_nnet.forward(wav)
# Or on GPU
continuous_nnet.cuda()
wav = wav.cuda()
out_wavs_gpu = continuous_nnet.forward(wav)